In [1]:
#import packages and modules
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import re

### script = 0-4
*script 1 groupsData

*script 2  minMaxPlayerStats

*script 3 shotsData

*script 4  matchData


### 1. Player data
Let's scrape Understat page for player with ID 1250 using BeatifulSoup

### 1.1 Player groupData

To get the json data from the Understat page we must first request, 
find it and execute the script we want , in this case number [1] , the groupData.



In [2]:
# Enter player ID link
link = "https://understat.com/player/1250"
res = requests.get(link)
soup = BeautifulSoup(res.content,'lxml')
scripts = soup.find_all('script')

# Get the groupData,
strings = scripts[1].string

# Getting rid of unnecessary characters from json data
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')
data = json.loads(json_data)

In [3]:
# We can call several keys and create dataframes from them
player_season_df = pd.DataFrame(data['season'])
player_season_df

,position,games,goals,shots,time,xG,assists,xA,key_passes,season,team,yellow,red,npg,npxG,xGChain,xGBuildup
0,FWR,35,23,139,2757,24.36429724097252,13,9.79082047007978,62,2021,Liverpool,1,0,18,19.797284230589867,32.158807165920734,11.22198199853301
1,FWR,37,22,126,3085,20.250846637412906,5,6.528526101261377,55,2020,Liverpool,0,0,16,15.683833638206124,28.968233548104763,9.800235655158758
2,FWR,34,19,132,2904,20.66331870108843,10,8.72604252398014,60,2019,Liverpool,1,0,16,18.379812128841877,31.37419793009758,8.42502685263753
3,FWR,38,22,137,3274,21.79245018400252,8,10.468589510768652,68,2018,Liverpool,1,0,19,19.508943611755967,31.780298396945,7.865945274010301
4,FWR,36,32,143,2954,25.136502970010042,10,8.344477602280676,62,2017,Liverpool,1,0,31,23.6141653098166,35.301465447992086,5.898578152060509
5,FWR,31,15,80,2498,13.411626555025578,11,9.63362380117178,71,2016,Roma,2,0,15,13.411626555025578,29.46748649328947,9.299841643311083
6,FWL,34,14,72,2753,9.377116854302585,6,7.0688703786581755,63,2015,Roma,1,1,14,9.377116854302585,23.043303050100803,10.797392231412232
7,Sub,16,6,29,909,3.814865604043007,3,1.9356205612421036,30,2014,Fiorentina,0,0,6,3.814865604043007,7.404609017074108,2.628024946898222
8,Sub,3,0,2,23,0.502302423119545,0,0.02637448161840439,1,2014,Chelsea,0,0,0,0.502302423119545,0.5454844385385513,0.01680753380060196


For some keys such as shotZones, shotTypes and OpenPlays , they are nested , meaning they have dictionaries inside dictionaries and if we proceed to create dataframes with them , we'll obtain dataframes with dictionaries inside them, this is a good example 

In [4]:
shot_type_df = pd.DataFrame(data['shotTypes'])
shot_type_df

,2014,2015,2016,2017,2018,2019,2020,2021
LeftFoot,"{'shotTypes': 'LeftFoot', 'season': '2014', 'g...","{'shotTypes': 'LeftFoot', 'season': '2015', 'g...","{'shotTypes': 'LeftFoot', 'season': '2016', 'g...","{'shotTypes': 'LeftFoot', 'season': '2017', 'g...","{'shotTypes': 'LeftFoot', 'season': '2018', 'g...","{'shotTypes': 'LeftFoot', 'season': '2019', 'g...","{'shotTypes': 'LeftFoot', 'season': '2020', 'g...","{'shotTypes': 'LeftFoot', 'season': '2021', 'g..."
RightFoot,"{'shotTypes': 'RightFoot', 'season': '2014', '...","{'shotTypes': 'RightFoot', 'season': '2015', '...","{'shotTypes': 'RightFoot', 'season': '2016', '...","{'shotTypes': 'RightFoot', 'season': '2017', '...","{'shotTypes': 'RightFoot', 'season': '2018', '...","{'shotTypes': 'RightFoot', 'season': '2019', '...","{'shotTypes': 'RightFoot', 'season': '2020', '...","{'shotTypes': 'RightFoot', 'season': '2021', '..."
Head,"{'shotTypes': 'Head', 'season': '2014', 'goals...","{'shotTypes': 'Head', 'season': '2015', 'goals...","{'shotTypes': 'Head', 'season': '2016', 'goals...","{'shotTypes': 'Head', 'season': '2017', 'goals...","{'shotTypes': 'Head', 'season': '2018', 'goals...","{'shotTypes': 'Head', 'season': '2019', 'goals...","{'shotTypes': 'Head', 'season': '2020', 'goals...","{'shotTypes': 'Head', 'season': '2021', 'goals..."
OtherBodyPart,NaN,NaN,NaN,NaN,NaN,NaN,"{'shotTypes': 'OtherBodyPart', 'season': '2020...",NaN


To unnest them what we can do is to apply the series function and create more dataframes out of it

In [5]:
shot_type_df['2021'].apply(pd.Series)

,shotTypes,season,goals,shots,xG,assists,key_passes,xA,npg,npxG,0
LeftFoot,LeftFoot,2021,19,117,19.16143623366952,2,20,3.5894319247454405,14,14.594423208385706,NaN
RightFoot,RightFoot,2021,4,15,3.9769467562437057,9,36,5.814623856917024,4,3.9769467562437057,NaN
Head,Head,2021,0,7,1.2259144522249699,1,3,0.3867647545412183,0,1.2259144522249699,NaN
OtherBodyPart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.2 Player shotsData
We use the same procedure as we did for Player groupData except this time we access script [3]

In [6]:
# Enter player ID link
link = "https://understat.com/player/1250"
res = requests.get(link)
soup = BeautifulSoup(res.content,'lxml')
scripts = soup.find_all('script')

# Get the shotsData
strings = scripts[3].string

# Getting rid of unnecessary characters from json data
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')
data = json.loads(json_data)

In [7]:
# Create pandas dataframe
player_shots_df = pd.DataFrame(data) 

In [8]:
player_shots_df

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,season,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
0,15217,92,MissedShots,0.8230000305175781,0.36900001525878906,0.06236456334590912,Mohamed Salah,h,1250,OpenPlay,2014,LeftFoot,4720,Chelsea,Swansea,4,2,2014-09-13 15:00:00,Eden Hazard,Pass
1,20034,84,SavedShot,0.92,0.5320000076293945,0.43993785977363586,Mohamed Salah,a,1250,OpenPlay,2014,LeftFoot,4560,Tottenham,Chelsea,5,3,2015-01-01 17:30:00,Eden Hazard,Pass
2,46908,87,BlockedShot,0.8859999847412109,0.5659999847412109,0.09676001220941544,Mohamed Salah,h,1250,OpenPlay,2014,LeftFoot,4929,Fiorentina,Atalanta,3,2,2015-02-08 11:30:00,Mario Gomez,BallTouch
3,47192,0,SavedShot,0.9640000152587891,0.6880000305175781,0.04913141205906868,Mohamed Salah,a,1250,OpenPlay,2014,LeftFoot,4937,Sassuolo,Fiorentina,1,3,2015-02-14 17:00:00,David Pizarro,Chipped
4,47197,16,MissedShots,0.9,0.6769999694824219,0.06032022833824158,Mohamed Salah,a,1250,OpenPlay,2014,LeftFoot,4937,Sassuolo,Fiorentina,1,3,2015-02-14 17:00:00,Khouma Babacar,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,477921,78,BlockedShot,0.8419999694824218,0.3609999847412109,0.05832324177026749,Mohamed Salah,h,1250,OpenPlay,2021,LeftFoot,16753,Liverpool,Wolverhampton Wanderers,3,1,2022-05-22 15:00:00,Sadio Mané,Pass
856,477927,83,Goal,0.9780000305175781,0.5420000076293945,0.6919867992401123,Mohamed Salah,h,1250,FromCorner,2021,LeftFoot,16753,Liverpool,Wolverhampton Wanderers,3,1,2022-05-22 15:00:00,Joel Matip,HeadPass
857,477928,88,BlockedShot,0.9009999847412109,0.4759999847412109,0.14238668978214264,Mohamed Salah,h,1250,OpenPlay,2021,LeftFoot,16753,Liverpool,Wolverhampton Wanderers,3,1,2022-05-22 15:00:00,Trent Alexander-Arnold,Pass
858,477930,91,SavedShot,0.935,0.44299999237060544,0.06315740197896957,Mohamed Salah,h,1250,OpenPlay,2021,Head,16753,Liverpool,Wolverhampton Wanderers,3,1,2022-05-22 15:00:00,None,None


### 1.3 Player matchData
matchData is script [4] 

In [9]:
# Enter player ID link
link = "https://understat.com/player/1250"
res = requests.get(link)
soup = BeautifulSoup(res.content,'lxml')
scripts = soup.find_all('script')

# Get the matchData
strings = scripts[4].string

# Getting rid of unnecessary characters from json data
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')
data = json.loads(json_data)

In [10]:
match_data_df = pd.DataFrame(data)
match_data_df

,goals,shots,xG,time,position,h_team,a_team,h_goals,a_goals,date,id,season,roster_id,xA,assists,key_passes,npg,npxG,xGChain,xGBuildup
0,1,8,1.1674931049346924,27,Sub,Liverpool,Wolverhampton Wanderers,3,1,2022-05-22,16753,2021,537896,0,0,0,1,1.1674931049346924,0.5637868046760559,0.08828042447566986
1,0,1,0.1275438666343689,15,Sub,Aston Villa,Liverpool,1,2,2022-05-10,16705,2021,534613,0.06885790824890137,0,1,0,0.1275438666343689,0.19640177488327026,0
2,0,3,0.25395557284355164,90,FWR,Liverpool,Tottenham,1,1,2022-05-07,16728,2021,533099,0.00995028018951416,0,2,0,0.25395557284355164,0.2639058530330658,0
3,0,2,0.08411531150341034,19,Sub,Newcastle United,Liverpool,0,1,2022-04-30,16721,2021,530133,0.03914148733019829,0,1,0,0.08411531150341034,0.12325680255889893,0.03648705407977104
4,0,2,0.14063450694084167,90,FWR,Liverpool,Everton,2,0,2022-04-24,16708,2021,529602,0.3986106216907501,1,2,0,0.14063450694084167,0.48341262340545654,0.3845406174659729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,1,3,0.4587177038192749,82,FWL,Sassuolo,Fiorentina,1,3,2015-02-14,4937,2014,52698,0.2239132970571518,1,2,1,0.4587177038192749,0.7049935460090637,0.022362560033798218
260,0,1,0.09676001220941544,24,Sub,Fiorentina,Atalanta,3,2,2015-02-08,4929,2014,52354,0.2711809575557709,0,3,0,0.09676001220941544,0.27318066358566284,0.09676001220941544
261,0,1,0.43993785977363586,15,Sub,Tottenham,Chelsea,5,3,2015-01-01,4560,2014,22951,0.02637448161840439,0,1,0,0.43993785977363586,0.4831198751926422,0.01680753380060196
262,0,0,0,2,Sub,Crystal Palace,Chelsea,1,2,2014-10-18,4702,2014,18957,0,0,0,0,0,0,0


### 1.4 Player Radar Data
minMaxPlayerStats = [2]

In [11]:
# Enter player ID link
link = "https://understat.com/player/1250"
res = requests.get(link)
soup = BeautifulSoup(res.content,'lxml')
scripts = soup.find_all('script')

# Get the minMaxPlayerStats
strings = scripts[2].string

# Getting rid of unnecessary characters from json data
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')
data = json.loads(json_data)

In [12]:
radar_df = pd.DataFrame(data['FW']).T
radar_df

,min,max,avg
goals,0.001100,0.012600,0.004200
xG,0.001728,0.012082,0.004155
shots,0.015000,0.073700,0.028000
assists,0.000000,0.004800,0.001400
xA,0.000264,0.005382,0.001316
key_passes,0.003600,0.030900,0.012000
xGChain,0.002727,0.016914,0.005338
xGBuildup,0.000243,0.006713,0.001318


### 2.0 Match Data
We scrape Understat for match ID 16753

### 2.1. Shot Data
*The second script available [1] contains shotsData.

*This will give us shot data for both home and away teams

In [13]:
# Entering Liverpool Vs Wolverhampton Wanderers match link
link = "https://understat.com/match/16753"
res = requests.get(link)
soup = BeautifulSoup(res.content,'lxml')
scripts = soup.find_all('script')

# Get the shotsData
strings = scripts[1].string 

# Getting rid of unnecessary characters from json data
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')
data = json.loads(json_data)

As it's happened before the shots data for the match is nested into home and away "h","a". To unnest them we must create to different DataFrames and merge them together

In [14]:
df_h = pd.DataFrame(data['h'])
df_a = pd.DataFrame(data['a'])
df = pd.concat([df_h, df_a], axis=1)
df

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,...,season,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
0,477897,4,MissedShots,0.875,0.47400001525878904,0.0188495721668005,Joel Matip,h,332,OpenPlay,...,2021,LeftFoot,16753,Liverpool,Wolverhampton Wanderers,3,1,2022-05-22 15:00:00,Raúl Jiménez,Pass
1,477898,9,MissedShots,0.9119999694824219,0.5990000152587891,0.274485319852829,Luis Díaz,h,10408,OpenPlay,...,2021,LeftFoot,16753,Liverpool,Wolverhampton Wanderers,3,1,2022-05-22 15:00:00,Pedro Neto,Pass
2,477899,12,BlockedShot,0.924000015258789,0.43099998474121093,0.08718594908714294,Naby Keita,h,5247,OpenPlay,...,2021,RightFoot,16753,Liverpool,Wolverhampton Wanderers,3,1,2022-05-22 15:00:00,Jonny,Chipped
3,477900,13,MissedShots,0.9490000152587891,0.57,0.04879196360707283,Joel Matip,h,332,FromCorner,...,2021,RightFoot,16753,Liverpool,Wolverhampton Wanderers,3,1,2022-05-22 15:00:00,Rúben Neves,Pass
4,477901,14,BlockedShot,0.8730000305175781,0.47900001525878905,0.10000427067279816,Diogo Jota,h,6854,OpenPlay,...,2021,Head,16753,Liverpool,Wolverhampton Wanderers,3,1,2022-05-22 15:00:00,Rúben Neves,Chipped
5,477903,23,Goal,0.8830000305175781,0.585,0.34907057881355286,Sadio Mané,h,838,OpenPlay,...,2021,LeftFoot,16753,Liverpool,Wolverhampton Wanderers,3,1,2022-05-22 15:00:00,Raúl Jiménez,Pass
6,477904,27,MissedShots,0.8440000152587891,0.335,0.04349058121442795,Trent Alexander-Arnold,h,1791,OpenPlay,...,2021,RightFoot,16753,Liverpool,Wolverhampton Wanderers,3,1,2022-05-22 15:00:00,João Moutinho,Pass
7,477905,27,MissedShots,0.93,0.6730000305175782,0.0634002611041069,Andrew Robertson,h,1688,OpenPlay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,477906,31,BlockedShot,0.784000015258789,0.5990000152587891,0.029489345848560333,Thiago Alcántara,h,229,OpenPlay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,477907,31,MissedShots,0.7319999694824219,0.6379999923706055,0.017833130434155464,Thiago Alcántara,h,229,OpenPlay,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 2.2 Line up Data
Now let's scrap the Team Sheet data for both teams:

In [15]:
# Entering Liverpool Vs Wolverhampton Wanderers match link
link = "https://understat.com/match/16753"
res = requests.get(link)
soup = BeautifulSoup(res.content,'lxml')
scripts = soup.find_all('script')

# Get the line up
strings = scripts[2].string 

# Getting rid of unnecessary characters from json data
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')
data = json.loads(json_data)

In [16]:
# Let's focus on the home team Liverpool
df = pd.DataFrame(data['h']).T
df

,id,goals,own_goals,shots,xG,time,player_id,team_id,position,player,...,yellow_card,red_card,roster_in,roster_out,key_passes,assists,xA,xGChain,xGBuildup,positionOrder
537884,537884,0,0,0,0,90,1257,87,GK,Alisson,...,0,0,0,0,0,0,0,0.24127297103405,0.24127297103405,1
537885,537885,0,0,3,0.14982256293296814,90,1791,87,DR,Trent Alexander-Arnold,...,0,0,0,0,6,0,0.4119507968425751,0.7344995141029358,0.47274237871170044,2
537886,537886,0,0,3,0.08162937313318253,90,332,87,DC,Joel Matip,...,1,0,0,0,2,1,0.9664720892906189,0.8362189531326294,0.5428840517997742,3
537887,537887,0,0,0,0,90,6326,87,DC,Ibrahima Konaté,...,0,0,0,0,0,0,0,0.9571263790130615,0.9571263790130615,3
537888,537888,1,0,2,0.535528838634491,90,1688,87,DL,Andrew Robertson,...,0,0,0,0,0,0,0,1.4529024362564087,1.3895021677017212,4
537889,537889,0,0,1,0.08718594908714294,75,5247,87,MC,Naby Keita,...,0,0,537895,0,1,0,0.10000427067279816,0.1871902197599411,0,9
537890,537890,0,0,2,0.04732247814536095,51,229,87,MC,Thiago Alcántara,...,0,0,537897,0,1,1,0.34907057881355286,0.6708783507347107,0.6235558986663818,9
537891,537891,0,0,0,0,90,605,87,MC,Jordan Henderson,...,0,0,0,0,1,0,0.0634002611041069,0.6848719716072083,0.6214717030525208,9
537892,537892,0,0,3,0.23374298214912415,63,6854,87,FWR,Diogo Jota,...,0,0,537896,0,0,0,0,0.2761821150779724,0.04243912547826767,14
537893,537893,1,0,3,0.42086783051490784,90,838,87,FW,Sadio Mané,...,0,0,0,0,4,0,0.21152955293655396,1.2983893156051636,0.6659919023513794,15


### 2.3 Match Stats
Here we can scrape some of the stats of the game such as goals scored, shots taken and received, ppdda, xG, xGA and interestingly enough win probability under the name of h_w , h_l and h_d

In [17]:
response = requests.get('https://understat.com/match/16753')
match_info = re.search("match_info\s+=\s+JSON.parse\('([^']+)", response.text)
decoded_string = bytes(match_info.groups()[0], 'utf-8').decode('unicode_escape')
matchObj = json.loads(decoded_string)
df = pd.DataFrame(pd.Series(matchObj))
df

,0
id,16753
fid,1549916
h,87
a,229
date,2022-05-22 15:00:00
league_id,1
season,2021
h_goals,3
a_goals,1
team_h,Liverpool


### 3.0 Get Fixtures

In [18]:
# Entering the league's  link
link = "https://understat.com/league/EPL"
res = requests.get(link)
soup = BeautifulSoup(res.content,'lxml')
scripts = soup.find_all('script')
# Get the fixtures data
strings = scripts[1].string 
# Getting rid of unnecessary characters from json data
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')
data = json.loads(json_data)

In [19]:
df = pd.DataFrame(data)
df

,id,isResult,h,a,goals,xG,datetime,forecast
0,16376,True,"{'id': '244', 'title': 'Brentford', 'short_tit...","{'id': '83', 'title': 'Arsenal', 'short_title'...","{'h': '2', 'a': '0'}","{'h': '1.88818', 'a': '1.02385'}",2021-08-13 19:00:00,"{'w': '0.6289', 'd': '0.2287', 'l': '0.1424'}"
1,16377,True,"{'id': '89', 'title': 'Manchester United', 'sh...","{'id': '245', 'title': 'Leeds', 'short_title':...","{'h': '5', 'a': '1'}","{'h': '1.64248', 'a': '0.568342'}",2021-08-14 11:30:00,"{'w': '0.6618', 'd': '0.2327', 'l': '0.1055'}"
2,16378,True,"{'id': '92', 'title': 'Burnley', 'short_title'...","{'id': '220', 'title': 'Brighton', 'short_titl...","{'h': '1', 'a': '2'}","{'h': '1.79548', 'a': '1.6853'}",2021-08-14 14:00:00,"{'w': '0.3894', 'd': '0.2877', 'l': '0.3229'}"
3,16379,True,"{'id': '80', 'title': 'Chelsea', 'short_title'...","{'id': '78', 'title': 'Crystal Palace', 'short...","{'h': '3', 'a': '0'}","{'h': '1.18709', 'a': '0.321701'}",2021-08-14 14:00:00,"{'w': '0.6405', 'd': '0.2822', 'l': '0.0773'}"
4,16380,True,"{'id': '72', 'title': 'Everton', 'short_title'...","{'id': '74', 'title': 'Southampton', 'short_ti...","{'h': '3', 'a': '1'}","{'h': '2.38863', 'a': '0.580601'}",2021-08-14 14:00:00,"{'w': '0.8359', 'd': '0.1234', 'l': '0.0407'}"
...,...,...,...,...,...,...,...,...
375,16751,True,"{'id': '78', 'title': 'Crystal Palace', 'short...","{'id': '89', 'title': 'Manchester United', 'sh...","{'h': '1', 'a': '0'}","{'h': '0.354618', 'a': '0.782005'}",2022-05-22 15:00:00,"{'w': '0.1466', 'd': '0.4028', 'l': '0.4506'}"
376,16752,True,"{'id': '75', 'title': 'Leicester', 'short_titl...","{'id': '74', 'title': 'Southampton', 'short_ti...","{'h': '4', 'a': '1'}","{'h': '3.42058', 'a': '1.05141'}",2022-05-22 15:00:00,"{'w': '0.9057', 'd': '0.0717', 'l': '0.0226'}"
377,16753,True,"{'id': '87', 'title': 'Liverpool', 'short_titl...","{'id': '229', 'title': 'Wolverhampton Wanderer...","{'h': '3', 'a': '1'}","{'h': '3.19385', 'a': '1.11708'}",2022-05-22 15:00:00,"{'w': '0.8332', 'd': '0.1119', 'l': '0.0549'}"
378,16754,True,"{'id': '88', 'title': 'Manchester City', 'shor...","{'id': '71', 'title': 'Aston Villa', 'short_ti...","{'h': '3', 'a': '2'}","{'h': '3.3209', 'a': '0.247841'}",2022-05-22 15:00:00,"{'w': '0.9766', 'd': '0.0197', 'l': '0.0037'}"


### 4. Get Table
Let's now scrape the standings table containing information about xG, xGA PPDA, etc

In [20]:
# Entering the league's  link
link = "https://understat.com/league/EPL"
res = requests.get(link)
soup = BeautifulSoup(res.content,'lxml')
scripts = soup.find_all('script')
# Get the table 
strings = scripts[2].string 
# Getting rid of unnecessary characters from json data
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')
data = json.loads(json_data)

In [21]:
df = pd.DataFrame(data.values())
df = df.explode("history")
h = df.pop("history")
df = pd.concat([df.reset_index(drop=True), pd.DataFrame(h.tolist())], axis=1)
df = df.infer_objects()

In [22]:
table = df.groupby(['title']).agg({'wins': 'sum', 'draws': 'sum', 'loses': 'sum', 'scored': 'sum', 
                                   'missed': 'sum', 'pts': 'sum', 'xG': 'sum', 'xGA': 'sum', 'xpts': 'sum', 
                                   'npxG': 'sum', 'npxGA': 'sum', 'deep': 'sum', 'deep_allowed': 'sum'}).reset_index()
table

,title,wins,draws,loses,scored,missed,pts,xG,xGA,xpts,npxG,npxGA,deep,deep_allowed
0,Arsenal,22,3,13,61,48,69,63.391577,48.389506,64.7622,57.191476,43.822492,314,220
1,Aston Villa,13,6,19,52,54,45,43.129543,53.095725,46.2012,40.846035,48.417956,258,258
2,Brentford,13,7,18,48,56,46,52.889575,49.500452,55.9965,48.322585,45.694635,232,226
3,Brighton,12,15,11,42,44,51,49.567475,47.211438,54.1822,44.239313,44.166798,305,244
4,Burnley,7,14,17,34,53,35,41.105891,60.223120,38.1035,39.583552,56.417304,184,316
5,Chelsea,21,11,6,76,33,74,69.297032,35.115624,73.3834,62.446512,30.548610,385,163
6,Crystal Palace,11,15,12,50,46,48,48.843907,41.331443,58.1500,42.663892,38.286775,213,238
7,Everton,11,6,21,43,66,39,44.469731,60.694247,43.5380,39.141610,59.171904,190,299
8,Leeds,9,11,18,42,79,38,51.186082,71.309612,43.8587,47.380234,67.503772,239,300
9,Leicester,14,10,14,62,59,52,49.488674,66.106594,42.3123,47.966335,59.063358,252,293


### 5. Getting all players stats
In this section we'll scrape the major stats for each player in la liga

In [23]:
# Entering the league's  link
link = "https://understat.com/league/EPL"
res = requests.get(link)
soup = BeautifulSoup(res.content,'lxml')
scripts = soup.find_all('script')
# Get the players stats 
strings = scripts[3].string 
# Getting rid of unnecessary characters from json data
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')
data = json.loads(json_data)
# Creating the dataframe
df = pd.DataFrame(data)
df

,id,player_name,games,time,goals,xG,assists,xA,shots,key_passes,yellow_cards,red_cards,position,team_title,npg,npxG,xGChain,xGBuildup
0,453,Son Heung-Min,35,3051,23,16.992173850536346,7,7.84695353731513,86,73,2,0,F M,Tottenham,23,16.992173850536346,29.62477569282055,10.343093157745898
1,1250,Mohamed Salah,35,2757,23,24.36429724097252,13,9.79082047007978,139,62,1,0,F S,Liverpool,18,19.797284230589867,32.158807165920734,11.22198199853301
2,2371,Cristiano Ronaldo,30,2468,18,17.212839871644974,3,4.424966126680374,110,26,8,0,F S,Manchester United,15,14.92933338880539,23.25371290743351,7.720640169456601
3,647,Harry Kane,37,3229,17,20.685065291821957,9,9.8182877227664,133,48,5,0,F M S,Tottenham,13,17.640390004962683,31.89218296855688,9.808405855670571
4,838,Sadio Mané,34,2833,16,16.82961391285062,2,5.269158786162734,98,39,5,0,F S,Liverpool,16,16.82961391285062,28.448093853890896,11.072969859465957
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,10327,Chiquinho,8,191,0,0.5611125975847244,3,1.0862634107470512,5,5,0,0,D S,Wolverhampton Wanderers,0,0.5611125975847244,1.7611463218927383,0.19358759000897408
533,10405,Josh Dasilva,9,242,0,0.101985864341259,0,0.18233786523342133,1,3,0,1,F M S,Brentford,0,0.101985864341259,0.44961406104266644,0.1652903314679861
534,10424,Daniel Chesters,1,1,0,0,0,0,0,0,0,0,S,West Ham,0,0,0,0
535,10525,Tony Springett,3,138,0,0.5181649327278137,0,0.514015793800354,1,2,1,0,M S,Norwich,0,0.5181649327278137,1.0925263166427612,0.06034556403756142


### References
https://www.kaggle.com/code/josegabrielgonzalez/understat-series-webscraping-using-beautiful-soup/notebook

https://www.youtube.com/watch?v=IsR5FrjNmro&t=232s